In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Read and Decode BBCI Data with Start-Stop-Markers

This tutorial shows how to read and decode BBCI data with start and stop markers. The data loading part is more complicated and it is advised to read the other tutorials before.

## Setup logging to see outputs

In [ ]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()


## Load and preprocess data

This is a bit more complicated than before since we have to add breaks etc. Here I now opt to add breaks do all preprocessings per run and only later combine them together.

In [ ]:
import numpy as np
from braindecode.datautil.splitters import concatenate_sets
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne, add_breaks
from braindecode.datasets.bbci import load_bbci_sets_from_folder
from copy import deepcopy
from braindecode.mne_ext.signalproc import resample_cnt, mne_apply
from braindecode.datautil.signalproc import lowpass_cnt
from braindecode.datautil.signalproc import exponential_running_standardize

def create_cnts(folder, runs,):
    # Load data
    cnts = load_bbci_sets_from_folder(folder, runs)
    
    # Now do some preprocessings:
    # Resampling to 250 Hz, lowpass below 38, eponential standardization
    
    new_cnts = []
    for cnt in cnts:
        # Only take some channels 
        #cnt = cnt.drop_channels(['STI 014']) # This would remove stimulus channel
        cnt = cnt.pick_channels(['C3', 'CPz', 'C4'])
        log.info("Preprocessing....")
        cnt = mne_apply(lambda a: lowpass_cnt(a, 38,cnt.info['sfreq'], axis=1), cnt)
        cnt = resample_cnt(cnt, 250)
        # mne apply will apply the function to the data (a 2d-numpy-array)
        # have to transpose data back and forth, since
        # exponential_running_standardize expects time x chans order
        # while mne object has chans x time order
        cnt = mne_apply(lambda a: exponential_running_standardize(
            a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
            cnt)
        new_cnts.append(cnt)
    return new_cnts

In [ ]:
from collections import OrderedDict

train_runs = [1,2,3]
train_cnts = create_cnts('/home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R01-8/', 
                         train_runs,)

name_to_start_code = OrderedDict([('Right Hand', 1), ('Feet', 4),
            ('Rotation', 8), ('Words', [10])])

name_to_stop_code = OrderedDict([('Right Hand', [20,21,22,23,24,28,30]),
        ('Feet', [20,21,22,23,24,28,30]),
        ('Rotation', [20,21,22,23,24,28,30]), 
        ('Words', [20,21,22,23,24,28,30])])


In [ ]:
test_runs = [9,10]
test_cnts = create_cnts('/home/schirrmr/data/robot-hall/AnLa/AnLaNBD1R09-10/', test_runs,)

## Create the model

We already create the model now, since we need to know the receptive field size for properly cutting out the data to predict. We need to cut out data starting at -receptive_field_size samples before the first sample we want to predict.

In [ ]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 650
in_chans = train_cnts[0].get_data().shape[0]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=5, input_time_length=input_time_length,
                        final_conv_length=29).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()
from braindecode.torch_ext.util import np_to_var
import numpy as np
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))
n_receptive_field = input_time_length - n_preds_per_input
receptive_field_ms = n_receptive_field * 1000.0 / train_cnts[0].info['sfreq']
print("Receptive field: {:d}/{:.2f} (samples/ms)".format(n_receptive_field,
                                                      receptive_field_ms))

### Create SignalAndTarget Sets

In [ ]:
from braindecode.datautil.trial_segment import create_signal_target_with_breaks_from_mne

break_start_offset_ms = 1000
break_stop_offset_ms = -500

train_sets = [create_signal_target_with_breaks_from_mne(
    cnt, name_to_start_code, [0,0], 
    name_to_stop_code, min_break_length_ms=1000, max_break_length_ms=10000,
    break_epoch_ival_ms=[500,-500],
    prepad_trials_to_n_samples=input_time_length) 
              for cnt in train_cnts]
train_set = concatenate_sets(train_sets)

In [ ]:
test_sets = [create_signal_target_with_breaks_from_mne(
    cnt, name_to_start_code, [0,0], 
    name_to_stop_code, min_break_length_ms=1000, max_break_length_ms=10000,
    break_epoch_ival_ms=[500,-500],
    prepad_trials_to_n_samples=input_time_length) 
              for cnt in test_cnts]
test_set = concatenate_sets(test_sets)

In [ ]:
from braindecode.datautil.splitters import split_into_two_sets

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)


## Setup optimizer and iterator

In [ ]:
from torch import optim

optimizer = optim.Adam(model.parameters())

In [ ]:
from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

## Setup Monitors, Loss function, Stop Criteria

In [ ]:
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression
from braindecode.torch_ext.losses import log_categorical_crossentropy


loss_function = log_categorical_crossentropy

model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

## Run experiment

In [ ]:
exp.run()

We arrive at about 54% accuracy. With only 3 sensors and 3 training runs, we cannot get much better :)